In [1]:
from config.configuration import RunDetails

# runDetail = RunDetails('config.yml', 'ASVspoof-2019_training')
# runDetail = RunDetails('config-mfcc.yml', 'ASVspoof-2019_training_mfcc')
runDetail = RunDetails('config.yml', 'ASVspoof-2019_small-eval-1')

notebookName = 'audio-deepfake-detection-training'

In [2]:
configFilename = runDetail.configFilename
runJobId = runDetail.jobId

In [3]:
import config.configuration as configuration
import model_definitions.model_cnn_definition as model_cnn_definition
from preprocessors.preprocessor_factory import PreprocessorFactory
from notebook_utils import notebookToPython
from processors.basic_model_training_processor import BasicModelTrainingProcessor
from processors.basic_model_evaluation_processor import BasicModelEvaluationProcessor

In [4]:
config = configuration.ConfigLoader(configFilename)

notebookToPython(notebookName)
job = config.getJobConfig(runJobId)

if (job.newModelGenerated == False):
    raise ValueError("This notebook is meant for training. Select a job without a value for 'persisted-model' set.")

Write python file
Generating new model name: output/ASVspoof-2019_small-eval-1_2025-04-01T12-13-55.954611.libjob
Assigned model name: output/ASVspoof-2019_small-eval-1_2025-04-01T12-13-55.954611.libjob


In [5]:
preproc_factory = PreprocessorFactory()
preprocessor = preproc_factory.newPreprocessor(job.preprocessor)


MelSpectrogramPreprocessor


In [6]:
X, y_encoded = preprocessor.extract_features_multipleSource(job, job.dataPathSuffix)

Loading C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_cm_protocols/ASVspoof2019.LA.cm.eval-small.trl.txt...
fullDataPath: C:/Users/tubas/workspace/Deepfake/data/ASVspoof-2019/LA/ASVspoof2019_LA_eval_small/flac
Loading audio files: 1
Loading audio files: 2
Loading audio files: 3
Loading audio files: 4
Number of audio files load: 4


In [7]:
trainingProc = BasicModelTrainingProcessor(job, model_cnn_definition.ModelCnnDefinition)
model, X_train, X_test, y_train, y_test = trainingProc.process(X, y_encoded, 1)

Selecting training and test data - traininSplitRandomState: 3
Training using 3 files.
    def buildModel(self) -> Model:
        print(f"__job__: {self.__job__}")
        input_shape = (self.__job__.numMels, self.width, self.channels)
        model_input = Input(shape = input_shape)
        job = self.__job__

        # Derived from:
        # Anagha, R., Arya, A., Narayan, V. H., Abhishek, S., & Anjali, T. (2023).
        #   Audio Deepfake Detection Using Deep Learning.
        #   2023 12th International Conference on System Modeling & Advancement in Research Trends (SMART), System Modeling & Advancement in Research Trends (SMART), 2023 12th International Conference On, 176–181.
        #   https://doi.org/10.1109/SMART59791.2023.10428163
        x = Conv2D(filters=32, kernel_size=job.kernelSize, activation='relu')(model_input)
        x = MaxPooling2D(pool_size=job.poolSize)(x)
        x = Conv2D(filters=64, kernel_size=job.kernelSize, activation='relu')(x)
        x = MaxPooling2D

### Test Model

In [8]:
evaluationProc = BasicModelEvaluationProcessor(job, model)
evaluationProc.process(y_encoded, X_test, y_test)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
y_pred: [[1.0000000e+00 3.1971816e-15]]
y_pred_work: [0]
true_labels: [[1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]
  Batches: 1 - Files: 1 - Score: 0.0 - Elements: 1


In [9]:
print("\n")
report = evaluationProc.reportSnapshot(trainingProc)
evaluationProc.writeReportToFile(job.persistedModelResults, report)